CS 418 Final Project
---
Using dataset scraped from Zillow
Includes all houses for sale in Chicago

data.csv is broken down into these columns...

| Address | Zip Code | Value | SQFT | Price Per SQFT | Number of Bedrooms | Number of Bathrooms | Number of Full Bathrooms | Year Built | Heating | Cooling | Parking | Type | Has Garage | Number of Storeis | HOA FEE |
|---------|----------|-------|------|----------------|--------------------|---------------------|--------------------------|------------|---------|---------|---------|------|------------|-------------------|---------|